https://realpython.com/pandas-project-gradebook/

In [69]:
from pathlib import Path
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os 
import glob 

# Loading data

- ROSTER and HW_EXAM_GRADES have common indices 
- TODO: try loading and manipulating with iterator


In [70]:
HERE = os.getcwd()
DATA_PATH = os.path.join(HERE, "data\\")

#Get roster information, using StudentID = NetID as index
ROSTER = pd.read_csv(
    (DATA_PATH + "roster.csv"),
    converters={"NetID": str.lower, "Email Address": str.lower},
    usecols=["Section", "Email Address", "NetID"],
    index_col="NetID",
)

#Get homework and exam grades for all students, using StudentID as index
#usage of lambda to filter out submission time 
HW_EXAM_GRADES = pd.read_csv(
    (DATA_PATH + "hw_exam_grades.csv"),
    converters={"SID": str.lower},
    usecols=lambda x: "Submission" not in x,
    index_col="SID",
)

#parse quiz grades from all 5 quiz_grades.csv, using email as index
QUIZ_GRADES = pd.DataFrame()
for file_path in glob.glob(DATA_PATH + "quiz_*_grades.csv"):
    quiz_name = " ".join(file_path.split("_")[:2])
    quiz_name = quiz_name.split("\\")[-1]
    quiz = pd.read_csv(
        file_path,
        converters={"Email": str.lower},
        index_col=["Email"],
        usecols=["Email", "Grade"],
    ).rename(columns={"Grade": quiz_name})
    QUIZ_GRADES = pd.concat([QUIZ_GRADES, quiz], axis=1)



# Data preparation / cleaning

- Merge all 3 dataframes into a single one containing all grades, indexed by StudentID
- Calculate grades for each student and final letter grade

In [72]:
#Merge and filling 
ALL_GRADES = pd.merge(ROSTER.merge(HW_EXAM_GRADES, left_index=True, right_index=True), QUIZ_GRADES, left_on='Email Address', right_index=True)
ALL_GRADES = ALL_GRADES.fillna(0)

# Variable initialization for calculation of final grade 
hw_pairs = dict()
HW_WEIGHT = 0.4

exam_pairs = dict()
EXAM_WEIGHT = {
    "Exam 1": 0.05,
    "Exam 2": 0.1,
    "Exam 3": 0.15
}

#Quiz max scores are not found within .csv files 
QUIZ_MAX_SCORES = {
    "quiz 1": 11,
    "quiz 2": 15,
    "quiz 3": 17,
    "quiz 4": 14,
    "quiz 5": 12 
}
QUIZ_WEIGHT = 0.3


# Calculating final grade
for column_number in range (4, 24, 2):
    hw_pairs[ALL_GRADES.columns[column_number]] = ALL_GRADES.columns[column_number + 1]

ALL_GRADES['Homework total'] = 0
for n_hw, n_hw_max in hw_pairs.items():
    ALL_GRADES['Homework total'] += (ALL_GRADES[n_hw] / ALL_GRADES[n_hw_max]) * 100 * HW_WEIGHT / 10

for column_number in range (24, 30, 2):
    exam_pairs[ALL_GRADES.columns[column_number]] = ALL_GRADES.columns[column_number + 1]

ALL_GRADES['Exam total'] = 0
for n_exam, n_exam_max in exam_pairs.items():
    ALL_GRADES['Exam total'] += (ALL_GRADES[n_exam] / ALL_GRADES[n_exam_max]) * 100 * EXAM_WEIGHT[n_exam] 

ALL_GRADES['Quiz total'] = 0
for n_quiz, n_quiz_max in QUIZ_MAX_SCORES.items():
    ALL_GRADES['Quiz total'] += (ALL_GRADES[n_quiz] / n_quiz_max) * 100 * QUIZ_WEIGHT / 5

ALL_GRADES['Final grade'] = (ALL_GRADES['Homework total'] + ALL_GRADES['Exam total'] + ALL_GRADES['Quiz total']).round(decimals=0) 

def to_letter_grade(grade: int) -> str:
    """
    Convert numerical grade to letter grade 

    args: numerical grade

    return:  letter grades A to F
    """
    if(grade > 89):
        return "A"
    elif(grade > 79):
        return "B"
    elif(grade > 69):
        return "C"
    elif(grade > 59):
        return "D"
    else:
        return "F"

ALL_GRADES['Letter Grade'] = ALL_GRADES['Final grade'].apply(func=to_letter_grade)


# Visualization 

- Matplotlib 
- 

In [45]:
ALL_GRADES.columns[0]

'Email Address'